In [ ]:
!pip install wandb
import wandb
# Replace with your actual API key
api_key = "17dab9d1bbdc37c41831799a4b0b50d3e97400c5"
project_name='RBCS1'
# Login to Weights & Biases
wandb.login(key=api_key)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import gym
import random
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)
import base64
import io  # For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob
import datetime

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

In [ ]:
env = gym.make('CartPole-v1')
env.seed(0)
state_shape = env.observation_space.shape[0]
action_shape = env.action_space.n
# print('observation space:', env.observation_space)
# print('action space:', env.action_space)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [ ]:
class Policy(nn.Module):
    def __init__(self, state_size=4, action_size=2, hidden_size=32):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        # we just consider 1 dimensional probability of action
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        if state.shape[1] != 4:  # Check if the state tensor has the expected shape
            state = state.view(1, 4)  # Reshape the state tensor to (1, 4)
        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)

In [ ]:
class Baseline(nn.Module):
    def __init__(self, state_size, hidden_size=32):
        super(Baseline, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        return x

In [ ]:
def reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for e in range(1, n_episodes):
        saved_log_probs = []
        rewards = []
        states = []
        state = env.reset()
        for t in range(max_t):
            states.append(state)
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        R = sum([a * b for a, b in zip(discounts, rewards)])

        # Calculate the baseline
        baseline_values = []
        for state_in_trajectory in states:
            state_tensor = torch.from_numpy(state_in_trajectory).float().unsqueeze(0).to(device)
            baseline_value = baseline(state_tensor)
            baseline_values.append(baseline_value)
        baseline_values = torch.cat(baseline_values, dim=0).requires_grad_()

        # Calculate the policy loss
        policy_loss = []
        for log_prob, baseline_value in zip(saved_log_probs, baseline_values):
            policy_loss.append(-(log_prob * (R - baseline_value)))
        policy_loss = torch.cat(policy_loss).sum()

        # Update the policy
        optimizer.zero_grad()
        policy_loss.backward(retain_graph=True)  # Retain the computation graph
        optimizer.step()

        # Update the baseline
        baseline_optimizer.zero_grad()
        baseline_loss = ((baseline_values - R) ** 2).mean()
        baseline_loss.backward()
        baseline_optimizer.step()

        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
        if np.mean(scores_deque) >= 195.0:
            print(f'Environment {env.unwrapped.spec.id} solved in {e - 100:d} episodes!\tAverage Score: {np.mean(scores_deque):.2f}')
            break

    return scores


# policy = Policy().to(device)
# optimizer = optim.Adam(policy.parameters(), lr=1e-2)
# baseline = Baseline(state_size=env.observation_space.shape[0]).to(device)
# baseline_optimizer = optim.Adam(baseline.parameters(), lr=1e-2)
# scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=2000)
# wandb.finish()  # Finish wandb run

In [ ]:
wandb.init(project=project_name)

wandb: Currently logged in as: rishi1906 (iitm_aero). Use `wandb login --relogin` to force relogin


In [ ]:
# wandb.init(project="R_CartPole_baseline", config={  # Initialize wandb first
#     "hidden_size": 32,  # Default values
#     "n_episodes": 1000,
#     "max_t": 1000,
#     "gamma": 0.99,
#     "print_every": 100,
#     "lr": 1e-2,
# })

# config = {
#     "hidden_size": wandb.config.hidden_size,  # Access wandb.config after initialization
#     "n_episodes": wandb.config.n_episodes,
#     "max_t": wandb.config.max_t,
#     "gamma": wandb.config.gamma,
#     "print_every": wandb.config.print_every,
#     "lr": wandb.config.lr,
# }


In [ ]:
# print(env.spec.reward_threshold)

In [ ]:
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'Average Reward',
      'goal': 'maximize'
    },
    'parameters': {
        'state_size': {
            'values': [state_shape]
        },
        'action_size': {
            'values': [action_shape]
        },
        'hidden_size': {
            'values': [32,64,128]
        },
        'max_t': {
            'values': [500, 1000]
        },
        'lr': {
            'values': [1e-2, 1e-3]
        },
        'baseline_lr': {
            'values': [1e-2, 1e-3]
        },
        'gamma': {
            'values': [0.99]
        },
        'alpha': {
            'values': [0.001, 0.0001]
        },
        'n_episodes': {
            'values': [2000]
        },
         "print_every": {
             'values': [100]
         }
    }
}

In [ ]:
# sweep_config = {
#     'method': 'grid',
#     'metric': {
#       'name': 'Average Reward',
#       'goal': 'maximize'
#     },
#     'parameters': {
#         'hidden_size': {
#             'values': [32,64,128]
#         },
#         'max_t': {
#             'values': [500, 1000, 1500]
#         },
#         'lr': {
#             'values': [1e-5, 1e-3,1e-4]
#         }
#     }
# }

In [ ]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config,  project=project_name)
max_sweep_run = 10 #update it

Create sweep with ID: h6mmiigu
Sweep URL: https://wandb.ai/iitm_aero/RBCS1/sweeps/h6mmiigu


In [ ]:
# def train(config=None):
#     # Set default values for hyperparameters
#     default_config = {
#         "hidden_size": 32,
#         "n_episodes": 1000,
#         "max_t": 1000,
#         "gamma": 1.0,
#         "print_every": 100,
#         "lr": 1e-2,
#         "baseline_lr": 1e-2,
#     }

#     # Initialize a new wandb run
#     run = wandb.init(config=config, reinit=True)

#     # If config is None, use the default values
#     if config is None:
#         run.config.update(default_config, allow_val_change=True)
#     config = run.config

#     # Initialize the environment and seed
#     env = gym.make('CartPole-v1')
#     env.seed(0)

#     # Create the policy network with the specified hyperparameters
#     policy = Policy(state_size=4, action_size=2, hidden_size=config.hidden_size).to(device)
#     optimizer = optim.Adam(policy.parameters(), lr=config.lr)

#     # Create the baseline network with the specified hyperparameters
#     baseline = Baseline(state_size=4).to(device)
#     baseline_optimizer = optim.Adam(baseline.parameters(), lr=config.baseline_lr)

#     # Run the REINFORCE algorithm with the specified hyperparameters
#     scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)

#     # Log the final score as a summary metric
#     run.summary["final_score"] = np.mean(scores[-100:])

#     # Finish the wandb run
#     run.finish()

#     # Return any necessary values or metrics
#     return np.mean(scores[-100:])

In [ ]:
def main():
    with wandb.init() as run:
        # Get the hyperparameters for this run
        config = wandb.config
        # # Initialize the environment and seed
        # env = gym.make('Acrobot-v1')
        # env.seed(0)

        # # Get the state and action sizes for the Acrobot-v1 environment
        # state_size = env.observation_space.shape[0]  # 6 dimensions
        # action_size = env.action_space.n  # 3 actions

        # Create the policy network with the specified hyperparameters
        policy = Policy(state_size=state_shape, action_size=action_shape, hidden_size=config.hidden_size).to(device)
        optimizer = optim.Adam(policy.parameters(), lr=config.lr)

        # Create the baseline network with the specified hyperparameters
        baseline = Baseline(state_size=state_shape).to(device)
        baseline_optimizer = optim.Adam(baseline.parameters(), lr=config.baseline_lr)

        # Run the REINFORCE algorithm with the specified hyperparameters
        scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)

        # # Check if the environment is solved
        # if np.mean(scores[-100:]) >= -100:  # Adjust the threshold as needed
        #     print(f'Environment {env.unwrapped.spec.id} solved in {e - 100:d} episodes!\tAverage Score: {np.mean(scores_deque):.2f}')





        # Log the final score as a summary metric
        run.summary["final_score"] = np.mean(scores[-100:])

        # # Finish the wandb run
        # run.finish()

        # # Return any necessary values or metrics
        # return np.mean(scores[-100:])

In [ ]:
wandb.agent(sweep_id, main, count=max_sweep_run)